## создание сводных для кажого бренда и фасовки, нужно сделать вычисления в этой таблице и соединить их вместе

In [35]:
import pandas as pd

### Бренды

In [36]:
brands = [
    'BlackBurn',
    'Burn',
    'B3',
    'Banger',
    'PeterRalf',
    'Overdose'
]

### Планы на каждый бренд

In [37]:
BB_Plan = 45000
B_Plan = 2019
B3_Plan = 201
BNG_Plan = 2925
PR_Plan = 40
OVD_Plan = 5392

### Вычисление доли фасовок и вычисление плана

In [38]:
df = pd.read_excel(r"D:\Analysis Burn\Кирилл Кузема\ABC.xlsx", sheet_name='Исходные данные')

In [39]:
# создание для каждого бренда таблицы с долей граммовки
for i in brands:
    exec(f"{i} = df.loc[df['Тип'] == '{i}'].groupby( \
        by=['Вес (гр)'] \
    )['Кг'].sum().reset_index()" )

    exec(f"{i}['доля'] = {i}['Кг'] / {i}['Кг'].sum() ")

    
    if i == 'BlackBurn':
        exec(f"{i}['планируемый_объем'] = {i}['доля'] * {BB_Plan}")

    if i == 'Burn':
        exec(f"{i}['планируемый_объем'] = {i}['доля'] * {B_Plan}")

    if i == 'B3':
        exec(f"{i}['планируемый_объем'] = {i}['доля'] * {B3_Plan}")

    if i == 'Banger':
        exec(f"{i}['планируемый_объем'] = {i}['доля'] * {BNG_Plan}")
        
    if i == 'PeterRalf':
        exec(f"{i}['планируемый_объем'] = {i}['доля'] * {PR_Plan}")

    if i == 'Overdose':
        exec(f"{i}['планируемый_объем'] = {i}['доля'] * {OVD_Plan}")


### создание таблиц для каждого бренда каждой фасовки

In [40]:
brands_dict = {
    'BlackBurn' : BlackBurn, 
    'Burn' : Burn,
    'B3' : B3, 
    'Banger' : Banger,
    'PeterRalf': PeterRalf, 
    'Overdose': Overdose
}

In [41]:
dfs = {}
for brand in brands:
   data = brands_dict[brand]
   for idx, row in data.iterrows():
      need_row = int(row['Вес (гр)'])
      dfs[f"{brand}_{str(need_row)}"] = df.loc[(df['Тип'] == brand) & (df['Вес (гр)'] == need_row)]

### Создание таблицы с планированием

In [42]:
pivot_tables = {}

for data in dfs:

    dfs.get(data)['year'] = dfs.get(data).loc[:, 'Месяц'].dt.year
    dfs.get(data)['month'] = dfs.get(data).loc[:, 'Месяц'].dt.month

    pivot_tables[data] = dfs.get(data).pivot_table(
        values=['Кг'],
        index=['Тип', 'Вес (гр)','Товар\вкус'],
        columns=['year', 'month'], 
        aggfunc="sum"
    ).round().reset_index()


    current_brand = pivot_tables[data]['Тип'][0]
    current_group = pivot_tables[data]['Вес (гр)'][0]

    for br in brands_dict:
        if br == current_brand:
            current_plan = brands_dict[br].loc[brands_dict[br]['Вес (гр)'] == current_group]['планируемый_объем'].iloc[0]
    

    # сделать двухэтажные заголовки обычными
    pivot_tables[data].columns = ['_'.join([str(x) for x in idx if x and x != 'Кг']) for idx in pivot_tables[data].columns]

    pivot_tables[data]['Последний месяц'] = pivot_tables[data].iloc[:,-1:]

    # добавить колонку Средний объем в месяц при наличии
    pivot_tables[data]['Средний объем в месяц при наличии'] = pivot_tables[data].median(axis=1, numeric_only=True)

    # добавить колонку Средний объем в месяц при наличии
    pivot_tables[data]['Наибольший объем в месяц'] = pivot_tables[data].max(axis=1, numeric_only=True)

    # добавить колонку Доля продаж исходя из ср-зн
    pivot_tables[data]['Доля продаж исходя из ср-зн'] = ( pivot_tables[data]['Средний объем в месяц при наличии'] / pivot_tables[data]['Средний объем в месяц при наличии'].sum() )

    # добавить колонку Планируемый объем
    pivot_tables[data]['Планируемый объем'] = current_plan * pivot_tables[data]['Доля продаж исходя из ср-зн']
    
    # добавить колонку Дополнительно
    pivot_tables[data]['Дополнительно'] = pivot_tables[data]['Планируемый объем'] * 0.1
    
    # добавить колонку Итоговый заказ на месяц
    pivot_tables[data]['Итоговый заказ на месяц'] = pivot_tables[data]['Дополнительно'] + pivot_tables[data]['Планируемый объем']

    # добавить колонку Итоговый заказ на следующий месяц
    pivot_tables[data]['Итоговый заказ на месяц вперед'] = pivot_tables[data]['Итоговый заказ на месяц'] * 1.1

    # добавить колонку Итоговый заказ на два месяца вперед
    pivot_tables[data]['Итоговый заказ на два месяца вперед'] = pivot_tables[data]['Итоговый заказ на месяц вперед'] * 1.1

    # добавить колонку Разница по отношению к прошлому месяцу 
    pivot_tables[data]['Разница по отношению к прошлому месяцу'] = pivot_tables[data]['Итоговый заказ на месяц'] - pivot_tables[data]['Последний месяц']

    pivot_tables[data].sort_values(by=['Доля продаж исходя из ср-зн'], ascending = False, inplace=True)

    pivot_tables[data]['Доля продаж исходя из ср-зн'] = pivot_tables[data]['Доля продаж исходя из ср-зн'] * 100

    pivot_tables[data].iloc[:,-10:] = pivot_tables[data].iloc[:,-10:].round(2)
    

C:\Users\alewk\AppData\Local\Temp\ipykernel_10812\4139720844.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs.get(data)['year'] = dfs.get(data).loc[:, 'Месяц'].dt.year
C:\Users\alewk\AppData\Local\Temp\ipykernel_10812\4139720844.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs.get(data)['month'] = dfs.get(data).loc[:, 'Месяц'].dt.month
C:\Users\alewk\AppData\Local\Temp\ipykernel_10812\4139720844.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

In [43]:
prepared_df = pd.DataFrame()
for i in pivot_tables:
    if prepared_df.empty == True:
        prepared_df = pivot_tables[i]
    else:
        prepared_df = pd.concat([prepared_df, pivot_tables[i]])

In [44]:
prepared_df.to_excel(r"D:\Analysis Burn\Кирилл Кузема\Планирование сентябрь_2023 готовая.xlsx")